In [1]:
!pip install torch_geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 245.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 474.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install pandas matplotlib gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 226.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 283.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 433.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 440.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

Cloning into 'hackaton'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 81 (delta 4), reused 4 (delta 4), pack-reused 70 (from 1)
Receiving objects: 100% (81/81), 105.83 MiB | 14.29 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Updating files: 100% (38/38), done.


In [4]:
%cd hackaton/

/workspace/hackaton


In [5]:
# !gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets
!gdown --folder https://drive.google.com/drive/folders/1vZVCPbux6brOnkfqil9CIj8a4iHlUwHV -O datasets

Retrieving folder contents
Retrieving folder 1oOxZgzm6GFEMqR9CE84syVkS3zDYm14G A
Processing file 1KK7CrI83nsEBG4ft_BgBeB3FPfrtqfs6 test.json.gz
Processing file 1HLDsT0NSttc9mI8obV4xsMdz0dXYsHg5 train.json.gz
Retrieving folder 1fZMQBg3Zkd9k8D3ExvMHLPDZ-A4Uldz3 B
Processing file 1nYi_Wj0c7FwMt1kNhenS5IOlfOAONUUO test.json.gz
Processing file 14bnCHnrPdLQ9fmLbnoi3GKFrwfHy2JPN train.json.gz
Retrieving folder 16I0ny5c6cuYncYTXNLro1i_bqUsJNeSc C
Processing file 1HftG39UqLt5Zu7sYHcDl8GN6PqZV7i_M test.json.gz
Processing file 1Q_QHBljWUS4ERBXbiZGQych-JKktRiMx train.json.gz
Retrieving folder 1ToGahOt-r0Llmo6Bwq6WHnzq31Y6ixwm D
Processing file 1yPgOhqJrmqP8eYfwCKgqdTz_blPZMzvj test.json.gz
Processing file 1oZy8-zEj8heFGWAJ2loD6adqpdARVMlA train.json.gz
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1KK7CrI83nsEBG4ft_BgBeB3FPfrtqfs6
From (redirected): https://drive.google.com/uc?

In [6]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from torch.optim.lr_scheduler import StepLR
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed()


### Data loader

In [7]:
import gzip
import json
import torch
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree

class GraphDatasetWithFeatures(Dataset):
    def __init__(self, filename, transform=None, pre_transform=None):
        self.raw = filename
        self.num_graphs, self.graphs_dicts = self._count_graphs()
        super().__init__(None, transform, pre_transform)

    def len(self):
        return self.num_graphs

    def get(self, idx):
        return dictToGraphObjectWithFeatures(self.graphs_dicts[idx])

    def _count_graphs(self):
        with gzip.open(self.raw, "rt", encoding="utf-8") as f:
            graphs_dicts = json.load(f)
            return len(graphs_dicts), graphs_dicts


def dictToGraphObjectWithFeatures(graph_dict):
    edge_index = torch.tensor(graph_dict["edge_index"], dtype=torch.long)
    edge_attr = torch.tensor(graph_dict["edge_attr"], dtype=torch.float) if graph_dict.get("edge_attr") else None
    num_nodes = graph_dict["num_nodes"]
    y = torch.tensor(graph_dict["y"][0], dtype=torch.long) if graph_dict.get("y") is not None else None

    node_features = None
    if edge_attr is not None:
        node_feature_dim = edge_attr.shape[1]
        node_features = torch.zeros((num_nodes, node_feature_dim), dtype=torch.float)

        row, col = edge_index
        node_features.index_add_(0, row, edge_attr)
        node_features.index_add_(0, col, edge_attr)

        deg = degree(edge_index[0], num_nodes=num_nodes, dtype=torch.float)
        node_features = node_features / deg.unsqueeze(1)

    graph_features = torch.tensor(graph_dict["graph_features"], dtype=torch.float) if graph_dict.get("graph_features") else None

    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, num_nodes=num_nodes, y=y, graph_x=graph_features)

### Model

In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINConv, GCNConv, global_add_pool

class GNN_node_Virtualnode_With_Features(torch.nn.Module):
    """
    Output:
        node representations
    """
    def __init__(self, num_layer, emb_dim, drop_ratio=0.5, JK="last", residual=False, gnn_type='gin', input_dim=None):
        '''
            emb_dim (int): node embedding dimensionality (output dimension of the first layer, and hidden dim for subsequent layers)
            input_dim (int): the actual dimension of the input node features (e.g., 7 for OGB-PPA derived features)
        '''
        super(GNN_node_Virtualnode_With_Features, self).__init__()
        self.num_layer = num_layer
        self.drop_ratio = drop_ratio
        self.JK = JK
        self.residual = residual
        self.emb_dim = emb_dim # This is the hidden dimension of the GNN layers

        if self.num_layer < 2:
            raise ValueError("Number of GNN layers must be greater than 1.")

        if input_dim is None:
            raise ValueError("input_dim must be provided for GNN_node_Virtualnode (e.g., 7 for OGB-PPA).")
        self.node_encoder = torch.nn.Linear(input_dim, emb_dim)

        ### set the initial virtual node embedding to 0.
        self.virtualnode_embedding = torch.nn.Embedding(1, emb_dim)
        torch.nn.init.constant_(self.virtualnode_embedding.weight.data, 0)

        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()

        self.mlp_virtualnode_list = torch.nn.ModuleList()

        for layer in range(num_layer):
            # PyG's GINConv and GCNConv expect `in_channels` and `out_channels`
            # For GINConv, it takes an MLP, so we pass `emb_dim` as the input/output dim of the MLP.
            # For GCNConv, it takes `in_channels` and `out_channels`.
            if gnn_type == 'gin':
                # The MLP passed to GINConv should map from emb_dim to emb_dim
                self.convs.append(GINConv(torch.nn.Sequential(
                    torch.nn.Linear(emb_dim, 2 * emb_dim),
                    torch.nn.BatchNorm1d(2 * emb_dim),
                    torch.nn.ReLU(),
                    torch.nn.Linear(2 * emb_dim, emb_dim)
                )))
            elif gnn_type == 'gcn':
                self.convs.append(GCNConv(emb_dim, emb_dim))
            else:
                raise ValueError('Undefined GNN type called {}'.format(gnn_type))

            self.batch_norms.append(torch.nn.BatchNorm1d(emb_dim))

        for layer in range(num_layer - 1):
            self.mlp_virtualnode_list.append(torch.nn.Sequential(
                torch.nn.Linear(emb_dim, 2*emb_dim),
                torch.nn.BatchNorm1d(2*emb_dim),
                torch.nn.ReLU(),
                torch.nn.Linear(2*emb_dim, emb_dim),
                torch.nn.BatchNorm1d(emb_dim),
                torch.nn.ReLU() # ReLU here is consistent with the original
            ))

    def forward(self, batched_data):
        x, edge_index, edge_attr, batch = batched_data.x, batched_data.edge_index, batched_data.edge_attr, batched_data.batch

        if x is None:
            raise ValueError("batched_data.x (node features) is None. Ensure your dataset correctly derives them.")
        h = self.node_encoder(x)
        h_list = [h]

        virtualnode_embedding = self.virtualnode_embedding(torch.zeros(batch[-1].item() + 1, dtype=torch.long, device=edge_index.device))

        for layer in range(self.num_layer):
            h_list[layer] = h_list[layer] + virtualnode_embedding[batch]

            if self.convs[layer].__class__.__name__ == 'GINConv':
                h = self.convs[layer](h_list[layer], edge_index) 
            elif self.convs[layer].__class__.__name__ == 'GCNConv':
                h = self.convs[layer](h_list[layer], edge_index, edge_weight=edge_attr if edge_attr is not None else None) # GCNConv can take edge_weight
            else:
                h = self.convs[layer](h_list[layer], edge_index, edge_attr) 

            h = self.batch_norms[layer](h)

            if layer == self.num_layer - 1:
                h = F.dropout(h, self.drop_ratio, training=self.training)
            else:
                h = F.dropout(F.relu(h), self.drop_ratio, training=self.training)

            if self.residual:
                h = h + h_list[layer] # Residual connection

            h_list.append(h)

            ### update the virtual nodes
            if layer < self.num_layer - 1:
                virtualnode_embedding_temp = global_add_pool(h_list[layer], batch) + virtualnode_embedding
                ### transform virtual nodes using MLP

                if self.residual:
                    virtualnode_embedding = virtualnode_embedding + F.dropout(self.mlp_virtualnode_list[layer](virtualnode_embedding_temp), self.drop_ratio, training=self.training)
                else:
                    virtualnode_embedding = F.dropout(self.mlp_virtualnode_list[layer](virtualnode_embedding_temp), self.drop_ratio, training=self.training)

        ### Different implementations of Jk-concat
        if self.JK == "last":
            node_representation = h_list[-1]
        elif self.JK == "sum":
            node_representation = 0
            # Iterate over all h_list (including initial embedding)
            for h_rep in h_list:
                node_representation += h_rep
        else:
            raise ValueError("Undefined JK type: {}".format(self.JK))

        return node_representation

In [9]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool, GlobalAttention, Set2Set
import torch.nn.functional as F
from torch_geometric.nn.inits import uniform

from src.conv import GNN_node, GNN_node_Virtualnode

class GNN_With_Features(torch.nn.Module):
    def __init__(self, num_class, num_layer=5, emb_dim=None, # <--- emb_dim can be dynamically set
                 gnn_type='gin', virtual_node=True, residual=False,
                 drop_ratio=0.5, JK="last", graph_pooling="mean",
                 num_graph_features=0):
        super(GNN_With_Features, self).__init__()
        self.num_layer = num_layer
        self.drop_ratio = drop_ratio
        self.JK = JK
        self.num_class = num_class
        self.graph_pooling = graph_pooling
        self.num_graph_features = num_graph_features

        if self.num_layer < 2:
            raise ValueError("Number of GNN layers must be greater than 1.")

        # If emb_dim is not provided, we will determine it from the first batch
        # This requires `self.gnn_node` to handle None initially or be initialized later
        # A safer approach is to pass the expected input dim to GNN_node
        # For OGB-PPA, the input node feature dimension is 7 (from edge_attr)
        self.input_node_feat_dim = 7 # <--- Explicitly set for OGB-PPA derived features
        self.emb_dim = emb_dim if emb_dim is not None else self.input_node_feat_dim # Use derived dim if not specified

        # GNN_node now needs to know the input dimension (self.input_node_feat_dim)
        if virtual_node:
            self.gnn_node = GNN_node_Virtualnode_With_Features(
                num_layer,
                self.emb_dim,
                JK=JK,
                drop_ratio=drop_ratio,
                residual=residual,
                gnn_type=gnn_type,
                input_dim=self.input_node_feat_dim) # <--- Pass input_dim
        else:
            self.gnn_node = GNN_node(num_layer, self.emb_dim, JK=JK, drop_ratio=drop_ratio, residual=residual, gnn_type=gnn_type,
                                    input_dim=self.input_node_feat_dim) # <--- Pass input_dim

        ### Pooling function
        if self.graph_pooling == "sum":
            self.pool = global_add_pool
        elif self.graph_pooling == "mean":
            self.pool = global_mean_pool
        elif self.graph_pooling == "max":
            self.pool = global_max_pool
        elif self.graph_pooling == "attention":
            # Gate network input dimension needs to match emb_dim
            self.pool = GlobalAttention(gate_nn = torch.nn.Sequential(torch.nn.Linear(self.emb_dim, 2*self.emb_dim), torch.nn.BatchNorm1d(2*self.emb_dim), torch.nn.ReLU(), torch.nn.Linear(2*self.emb_dim, 1)))
        elif self.graph_pooling == "set2set":
            self.pool = Set2Set(self.emb_dim, processing_steps = 2)
        else:
            raise ValueError("Invalid graph pooling type.")

        # Final linear layer
        if graph_pooling == "set2set":
            pooled_dim = 2 * self.emb_dim
        else:
            pooled_dim = self.emb_dim

        self.graph_pred_linear = torch.nn.Linear(pooled_dim + self.num_graph_features, self.num_class)


    def forward(self, batched_data):
        # The GNN_node module expects batched_data.x for node features
        # Your dictToGraphObject now correctly sets batched_data.x
        h_node = self.gnn_node(batched_data)
        h_graph = self.pool(h_node, batched_data.batch)

        if self.num_graph_features > 0:
            if not hasattr(batched_data, 'graph_x') or batched_data.graph_x is None:
                raise ValueError("num_graph_features is set > 0, but 'graph_x' attribute not found in batched_data.")
            if batched_data.graph_x.shape[0] != h_graph.shape[0]:
                raise ValueError(f"Batch size of graph_x ({batched_data.graph_x.shape[0]}) does not match pooled graph embeddings ({h_graph.shape[0]}).")

            h_graph = torch.cat([h_graph, batched_data.graph_x], dim=-1)

        return self.graph_pred_linear(h_graph)

### Loss

In [10]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class GeneralizedCrossEntropyLoss(nn.Module):
    """
    Generalized Cross Entropy Loss for Training Deep Neural Networks with Noisy Labels
    2018 - Zhang et al. - https://arxiv.org/pdf/1805.07836.pdf
    
    Args:
        q (float): Robustness parameter. q=0 gives standard cross-entropy, q=1 gives MAE.
                  Recommended range: [0.1, 0.7] for noise robustness.
        reduction (str): Specifies the reduction to apply to the output.
        eps (float): Small value to prevent numerical instability.
    """
    
    def __init__(self, q=0.3, reduction='mean', eps=1e-7):
        super(GeneralizedCrossEntropyLoss, self).__init__()
        self.q = q
        self.reduction = reduction
        self.eps = eps
    
    def forward(self, logits, targets):
        """
        Args:
            logits: Tensor of shape (N, C) containing raw logits
            targets: Tensor of shape (N,) containing class indices
        """
        # Convert logits to probabilities
        y_pred = F.softmax(logits, dim=-1)
        
        # Convert targets to one-hot encoding
        y_true = F.one_hot(targets, num_classes=logits.size(-1)).float()
        
        # Compute GCE loss with clamping for numerical stability
        # Clamp predictions to avoid numerical issues
        y_pred = torch.clamp(y_pred, min=self.eps, max=1.0 - self.eps)
        
        # Compute the sum of y_true * y_pred along class dimension
        pred_sum = torch.sum(y_true * y_pred, dim=-1)
        
        # Clamp again before taking the power to ensure stability
        pred_sum = torch.clamp(pred_sum, min=self.eps)
        
        # GCE loss: (1 - pred_sum^q) / q
        if self.q == 0:
            # Special case: standard cross-entropy (limit as q->0)
            loss = -torch.log(pred_sum)
        else:
            intermed = torch.pow(pred_sum, self.q)
            loss = (1 - intermed) / self.q
        
        if self.reduction == 'mean':
            return torch.mean(loss)
        elif self.reduction == 'sum':
            return torch.sum(loss)
        else:  # 'none'
            return loss

In [11]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch, scaler=None):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [35]:
def evaluate(data_loader, model, device, criterion=None, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                if criterion is not None:
                    total_loss += criterion(output, data.y).item()

            else:
                predictions.extend(pred.cpu().numpy())
    
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader), accuracy
        
    return predictions

In [13]:
def save_predictions(predictions, test_path):
    # script_dir = os.path.dirname(os.path.abspath(__file__))
    script_dir = os.getcwd()
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [14]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [15]:
# script_dir = os.path.dirname(os.path.abspath(__file__))
script_dir = os.getcwd()
# device = torch.device("cuda:1" if torch.cuda.is_available() else torch.device("cpu"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = 5

In [16]:
class Args():
    def __init__(
            self,
            dataset_name,
            noise_ratio=0.2,
            num_layer=5,
            emb_dim=256,
            drop_ratio=0.2,
            batch_size=32,
    ):
        self.train_path = f"datasets/{dataset_name}/train.json.gz"
        self.test_path = f"datasets/{dataset_name}/test.json.gz"

        self.noise_ratio = noise_ratio

        self.num_layer = num_layer
        self.emb_dim = emb_dim
        self.drop_ratio = drop_ratio
        self.batch_size = batch_size
        

In [17]:
args = Args("B")

In [18]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))

In [22]:
full_dataset = GraphDatasetWithFeatures(args.train_path)

In [19]:
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())  # Console output as well

In [20]:
# Define checkpoint path relative to the script's directory
checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)

In [23]:
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size

generator = torch.Generator().manual_seed(12)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=8, pin_memory=True)

In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool, GlobalAttention, Set2Set
from torch_geometric.nn.inits import uniform
import numpy as np

# 1. CONTRASTIVE LEARNING APPROACH
class GNNContrastive(torch.nn.Module):
    """GNN with contrastive learning for representation learning"""
    def __init__(self, num_class, num_layer=5, emb_dim=None, 
                 gnn_type='gin', virtual_node=True, residual=False,
                 drop_ratio=0.5, JK="last", graph_pooling="mean",
                 num_graph_features=0, projection_dim=128):
        super(GNNContrastive, self).__init__()
        
        # Copy your existing GNN initialization
        self.num_layer = num_layer
        self.drop_ratio = drop_ratio
        self.JK = JK
        self.num_class = num_class
        self.graph_pooling = graph_pooling
        self.num_graph_features = num_graph_features
        self.input_node_feat_dim = 7
        self.emb_dim = emb_dim if emb_dim is not None else self.input_node_feat_dim

        # Your existing GNN_node initialization
        from src.conv import GNN_node
        if virtual_node:
            self.gnn_node = GNN_node_Virtualnode_With_Features(
                num_layer,
                self.emb_dim,
                JK=JK,
                drop_ratio=drop_ratio,
                residual=residual,
                gnn_type=gnn_type,
                input_dim=self.input_node_feat_dim)
        else:
            self.gnn_node = GNN_node(num_layer, self.emb_dim, JK=JK, 
                                   drop_ratio=drop_ratio, residual=residual, 
                                   gnn_type=gnn_type, input_dim=self.input_node_feat_dim)

        # Pooling (same as your code)
        if self.graph_pooling == "sum":
            self.pool = global_add_pool
        elif self.graph_pooling == "mean":
            self.pool = global_mean_pool
        elif self.graph_pooling == "max":
            self.pool = global_max_pool
        elif self.graph_pooling == "attention":
            self.pool = GlobalAttention(gate_nn=torch.nn.Sequential(
                torch.nn.Linear(self.emb_dim, 2*self.emb_dim), 
                torch.nn.BatchNorm1d(2*self.emb_dim), 
                torch.nn.ReLU(), 
                torch.nn.Linear(2*self.emb_dim, 1)))
        elif self.graph_pooling == "set2set":
            self.pool = Set2Set(self.emb_dim, processing_steps=2)

        # Calculate pooled dimension
        if graph_pooling == "set2set":
            pooled_dim = 2 * self.emb_dim
        else:
            pooled_dim = self.emb_dim
            
        self.pooled_dim = pooled_dim + self.num_graph_features

        # Projection head for contrastive learning
        self.projection_head = nn.Sequential(
            nn.Linear(self.pooled_dim, self.pooled_dim),
            nn.ReLU(),
            nn.Linear(self.pooled_dim, projection_dim)
        )
        
        # Classification head
        self.classifier = nn.Linear(self.pooled_dim, self.num_class)
        
    def forward(self, batched_data, return_projection=False):
        """Forward pass with option to return projection for contrastive learning"""
        h_node = self.gnn_node(batched_data)
        h_graph = self.pool(h_node, batched_data.batch)

        if self.num_graph_features > 0:
            if not hasattr(batched_data, 'graph_x') or batched_data.graph_x is None:
                raise ValueError("num_graph_features is set > 0, but 'graph_x' attribute not found.")
            h_graph = torch.cat([h_graph, batched_data.graph_x], dim=-1)

        if return_projection:
            projection = F.normalize(self.projection_head(h_graph), dim=1)
            return h_graph, projection
        
        return self.classifier(h_graph)
    
    def get_representations(self, batched_data):
        """Get graph representations without classification"""
        h_node = self.gnn_node(batched_data)
        h_graph = self.pool(h_node, batched_data.batch)
        
        if self.num_graph_features > 0:
            if hasattr(batched_data, 'graph_x') and batched_data.graph_x is not None:
                h_graph = torch.cat([h_graph, batched_data.graph_x], dim=-1)
        
        return h_graph


# 2. NOISE-ROBUST LOSS FUNCTIONS
class SymmetricCrossEntropy(nn.Module):
    """Symmetric Cross-Entropy Loss for noisy labels"""
    def __init__(self, alpha=0.1, beta=1.0, num_classes=6):
        super(SymmetricCrossEntropy, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.num_classes = num_classes

    def forward(self, pred, labels):
        # Standard cross-entropy
        ce = F.cross_entropy(pred, labels, reduction='none')
        
        # Reverse cross-entropy
        pred_softmax = F.softmax(pred, dim=1)
        pred_softmax = torch.clamp(pred_softmax, min=1e-7, max=1.0)
        
        label_one_hot = F.one_hot(labels, self.num_classes).float()
        rce = -1 * torch.sum(pred_softmax * torch.log(label_one_hot + 1e-7), dim=1)
        
        # Combine both losses
        loss = self.alpha * ce + self.beta * rce
        return loss.mean()


class GeneralizedCrossEntropy(nn.Module):
    """Generalized Cross-Entropy Loss"""
    def __init__(self, q=0.7):
        super(GeneralizedCrossEntropy, self).__init__()
        self.q = q

    def forward(self, pred, labels):
        pred_softmax = F.softmax(pred, dim=1)
        pred_softmax = torch.clamp(pred_softmax, min=1e-7, max=1.0)
        
        label_one_hot = F.one_hot(labels, pred.shape[1]).float()
        loss = (1 - torch.sum(label_one_hot * pred_softmax**self.q, dim=1)) / self.q
        return loss.mean()


# 3. CONTRASTIVE LEARNING TRAINING
class ContrastiveLoss(nn.Module):
    """InfoNCE Loss for contrastive learning"""
    def __init__(self, temperature=0.1):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, features, labels=None):
        batch_size = features.shape[0]
        
        # Normalize features
        features = F.normalize(features, dim=1)
        
        # Compute similarity matrix
        similarity_matrix = torch.matmul(features, features.T) / self.temperature
        
        # Create positive pairs mask (same class)
        if labels is not None:
            labels = labels.contiguous().view(-1, 1)
            mask = torch.eq(labels, labels.T).float()
            # Remove diagonal (self-similarity)
            mask = mask.fill_diagonal_(0)
        else:
            # If no labels, create augmentation-based positive pairs
            # This requires data augmentation strategy
            mask = torch.eye(batch_size).to(features.device)
            mask = torch.cat([mask[batch_size//2:], mask[:batch_size//2]], dim=0)
        
        # Apply contrastive loss
        exp_sim = torch.exp(similarity_matrix)
        sum_exp_sim = exp_sim.sum(dim=1, keepdim=True)
        
        positive_sim = (exp_sim * mask).sum(dim=1)
        loss = -torch.log(positive_sim / (sum_exp_sim.squeeze() + 1e-8))
        
        return loss.mean()


# 4. TRAINING PIPELINE WITH REPRESENTATION LEARNING
class NoiseRobustTrainer:
    def __init__(self, model, device='cuda', warmup_epochs=10):
        self.model = model.to(device)
        self.device = device
        self.warmup_epochs = warmup_epochs
        
        # Different loss functions
        self.contrastive_loss = ContrastiveLoss()
        self.sce_loss = SymmetricCrossEntropy()
        self.gce_loss = GeneralizedCrossEntropy()
        
        # Optimizers
        self.optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
    def pretrain_representations(self, dataloader, epochs=50):
        """Pretrain with contrastive learning"""
        self.model.train()
        
        for epoch in range(epochs):
            total_loss = 0
            for batch in dataloader:
                batch = batch.to(self.device)
                
                # Get representations and projections
                representations, projections = self.model(batch, return_projection=True)
                
                # Contrastive loss (unsupervised)
                loss = self.contrastive_loss(projections)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                total_loss += loss.item()
            
            if epoch % 10 == 0:
                print(f'Pretrain Epoch {epoch}, Loss: {total_loss/len(dataloader):.4f}')
    
    def train_with_noise_robustness(self, dataloader, epochs=100):
        """Train with noise-robust losses"""
        self.model.train()
        
        for epoch in range(epochs):
            total_loss = 0
            
            for batch in dataloader:
                batch = batch.to(self.device)
                labels = batch.y
                
                # Forward pass
                logits = self.model(batch)
                
                # Choose loss based on epoch
                if epoch < self.warmup_epochs:
                    # Use symmetric cross-entropy for warmup
                    loss = self.sce_loss(logits, labels)
                else:
                    # Use generalized cross-entropy
                    loss = self.gce_loss(logits, labels)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                total_loss += loss.item()
            
            
            print(f'Train Epoch {epoch}, Loss: {total_loss/len(dataloader):.4f}')


# 5. SAMPLE SELECTION STRATEGY
class SampleSelector:
    def __init__(self, model, threshold_percentile=30):
        self.model = model
        self.threshold_percentile = threshold_percentile
    
    def select_clean_samples(self, dataloader):
        """Select samples with low loss (likely clean)"""
        self.model.eval()
        all_losses = []
        all_indices = []
        
        with torch.no_grad():
            for batch_idx, batch in enumerate(dataloader):
                batch = batch.to(next(self.model.parameters()).device)
                logits = self.model(batch)
                losses = F.cross_entropy(logits, batch.y, reduction='none')
                
                all_losses.extend(losses.cpu().numpy())
                # You'll need to track original indices
                all_indices.extend(range(batch_idx * batch.batch_size, 
                                       (batch_idx + 1) * batch.batch_size))
        
        # Select samples with losses below threshold
        threshold = np.percentile(all_losses, self.threshold_percentile)
        clean_indices = [idx for idx, loss in zip(all_indices, all_losses) 
                        if loss < threshold]
        
        return clean_indices



In [64]:
model = GNNContrastive(
    gnn_type='gin',
    num_class=6,
    num_layer=args.num_layer,
    emb_dim=args.emb_dim,
    drop_ratio=args.drop_ratio,
    virtual_node=True,
    residual=True,
).to(device)

In [65]:
trainer = NoiseRobustTrainer(model)

In [66]:

# Step 1: Pretrain representations
print("Pretraining representations...")
trainer.pretrain_representations(train_loader, epochs=30)


Pretraining representations...
Pretrain Epoch 0, Loss: 3.4731
Pretrain Epoch 10, Loss: 3.4658
Pretrain Epoch 20, Loss: 3.4658


In [67]:
# Step 2: Train with noise-robust losses
print("Training with noise-robust losses...")
trainer.train_with_noise_robustness(clean_train, epochs=20)

Training with noise-robust losses...
Train Epoch 0, Loss: 13.3444
Train Epoch 1, Loss: 12.2009
Train Epoch 2, Loss: 11.6601
Train Epoch 3, Loss: 12.1428
Train Epoch 4, Loss: 11.5368
Train Epoch 5, Loss: 11.2384
Train Epoch 6, Loss: 11.1259
Train Epoch 7, Loss: 11.5249
Train Epoch 8, Loss: 11.3281
Train Epoch 9, Loss: 10.9869
Train Epoch 10, Loss: 0.8713
Train Epoch 11, Loss: 0.8716
Train Epoch 12, Loss: 0.8399
Train Epoch 13, Loss: 0.8343
Train Epoch 14, Loss: 0.8395
Train Epoch 15, Loss: 0.8196
Train Epoch 16, Loss: 0.8107
Train Epoch 17, Loss: 0.8047
Train Epoch 18, Loss: 0.8056
Train Epoch 19, Loss: 0.7991


In [74]:
print("Training with noise-robust losses...")
trainer.train_with_noise_robustness(train_loader, epochs=100)

Training with noise-robust losses...
Train Epoch 0, Loss: 10.6582
Train Epoch 1, Loss: 10.4948
Train Epoch 2, Loss: 10.3348
Train Epoch 3, Loss: 10.0820
Train Epoch 4, Loss: 9.9448
Train Epoch 5, Loss: 9.8221
Train Epoch 6, Loss: 9.7185
Train Epoch 7, Loss: 9.6640
Train Epoch 8, Loss: 9.6264
Train Epoch 9, Loss: 9.5566
Train Epoch 10, Loss: 0.7630
Train Epoch 11, Loss: 0.7422
Train Epoch 12, Loss: 0.7421
Train Epoch 13, Loss: 0.7397
Train Epoch 14, Loss: 0.7339
Train Epoch 15, Loss: 0.7339
Train Epoch 16, Loss: 0.7345
Train Epoch 17, Loss: 0.7282
Train Epoch 18, Loss: 0.7264
Train Epoch 19, Loss: 0.7229
Train Epoch 20, Loss: 0.7245
Train Epoch 21, Loss: 0.7196
Train Epoch 22, Loss: 0.7221
Train Epoch 23, Loss: 0.7235
Train Epoch 24, Loss: 0.7199
Train Epoch 25, Loss: 0.7175
Train Epoch 26, Loss: 0.7132
Train Epoch 27, Loss: 0.7179
Train Epoch 28, Loss: 0.7194
Train Epoch 29, Loss: 0.7120
Train Epoch 30, Loss: 0.7191
Train Epoch 31, Loss: 0.7156
Train Epoch 32, Loss: 0.7129
Train Epoch 

In [73]:
# Step 3: Optional - Sample selection for refinement
selector = SampleSelector(model)

In [41]:
clean_indices_train = selector.select_clean_samples(train_loader)
print(f"Selected {len(clean_indices_train)} clean samples")

Selected 1315 clean samples


In [42]:
clean_indices_val = selector.select_clean_samples(val_loader)
print(f"Selected {len(clean_indices_val)} clean samples")

Selected 336 clean samples


In [46]:
from torch.utils.data import Subset

def create_subset_dataloader(dataset, clean_indices, batch_size=32, shuffle=True):
    """
    Create a new dataloader with only the selected clean samples
    
    Args:
        dataset: Your GraphDatasetWithFeatures instance
        clean_indices: List of indices for clean samples
        batch_size: Batch size for the dataloader
        shuffle: Whether to shuffle the data
    
    Returns:
        DataLoader with subset of clean samples
    """
    subset = Subset(dataset, clean_indices)
    return DataLoader(subset, batch_size=batch_size, shuffle=shuffle, num_workers=8, pin_memory=True)

In [47]:
clean_train = create_subset_dataloader(train_dataset, clean_indices_train, batch_size=args.batch_size, shuffle=True)
clean_val = create_subset_dataloader(val_dataset, clean_indices_val, batch_size=args.batch_size, shuffle=False)

In [80]:
model.load_state_dict(torch.load("/workspace/hackaton/checkpoints/model_B_best.pth_pretrained.pth"))

<All keys matched successfully>

In [84]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 376.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 369.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [96]:
import torch
import torch.nn.functional as F
import numpy as np
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset
from sklearn.metrics import accuracy_score, f1_score

class DivideMixGraphTrainer:
    """
    DivideMix-style training adapted for graph data with high noise rates
    Enhanced with validation and monitoring capabilities
    """
    def __init__(self, model1, model2, dataset, val_dataset=None, device='cuda', warmup_epochs=1):
        self.model1 = model1.to(device)
        self.model2 = model2.to(device)
        self.dataset = dataset
        self.val_dataset = val_dataset
        self.device = device
        self.warmup_epochs = warmup_epochs
        
        self.optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
        self.optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)
        
        # Track clean/noisy split for each model
        self.clean_indices1 = []
        self.clean_indices2 = []
        
        # Validation tracking
        self.train_history = {'loss': [], 'accuracy': []}
        self.val_history = {'loss': [], 'accuracy': [], 'f1': []}
        self.best_val_acc = 0.0
        self.best_models = {'model1': None, 'model2': None}
        
        # Early stopping
        self.patience = 20
        self.patience_counter = 0
        
    def evaluate_model(self, model, dataloader, return_predictions=False):
        """Evaluate model on given dataloader"""
        model.eval()
        total_loss = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for batch in dataloader:
                batch = batch.to(self.device)
                logits = model(batch)
                loss = F.cross_entropy(logits, batch.y)
                
                total_loss += loss.item()
                preds = torch.argmax(logits, dim=1)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(batch.y.cpu().numpy())
        
        avg_loss = total_loss / len(dataloader)
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')
        
        if return_predictions:
            return avg_loss, accuracy, f1, all_preds, all_labels
        return avg_loss, accuracy, f1
    
    def validate_models(self, val_dataloader, epoch):
        """Validate both models and track best performance"""
        if val_dataloader is None:
            return
        
        # Evaluate both models
        val_loss1, val_acc1, val_f1_1 = self.evaluate_model(self.model1, val_dataloader)
        val_loss2, val_acc2, val_f1_2 = self.evaluate_model(self.model2, val_dataloader)
        
        # Use ensemble average for validation metrics
        avg_val_loss = (val_loss1 + val_loss2) / 2
        avg_val_acc = (val_acc1 + val_acc2) / 2
        avg_val_f1 = (val_f1_1 + val_f1_2) / 2
        
        # Track validation history
        self.val_history['loss'].append(avg_val_loss)
        self.val_history['accuracy'].append(avg_val_acc)
        self.val_history['f1'].append(avg_val_f1)
        
        # Save best models
        if avg_val_acc > self.best_val_acc:
            self.best_val_acc = avg_val_acc
            self.best_models['model1'] = {
                'state_dict': self.model1.state_dict().copy(),
                'optimizer': self.optimizer1.state_dict().copy()
            }
            self.best_models['model2'] = {
                'state_dict': self.model2.state_dict().copy(),
                'optimizer': self.optimizer2.state_dict().copy()
            }
            self.patience_counter = 0
        else:
            self.patience_counter += 1
        
        print(f"Epoch {epoch} - Val Loss: {avg_val_loss:.4f}, "
              f"Val Acc: {avg_val_acc:.4f}, Val F1: {avg_val_f1:.4f}")
        
        return avg_val_acc
    
    def should_early_stop(self):
        """Check if early stopping criteria is met"""
        return self.patience_counter >= self.patience
    
    def load_best_models(self):
        """Load the best performing models"""
        if self.best_models['model1'] is not None:
            self.model1.load_state_dict(self.best_models['model1']['state_dict'])
            self.optimizer1.load_state_dict(self.best_models['model1']['optimizer'])
            
        if self.best_models['model2'] is not None:
            self.model2.load_state_dict(self.best_models['model2']['state_dict'])
            self.optimizer2.load_state_dict(self.best_models['model2']['optimizer'])
    
    def warmup_training(self, dataloader, val_dataloader=None, epochs=10):
        """Warmup phase with symmetric cross-entropy and validation"""
        print("Warmup training phase...")
        
        for epoch in range(epochs):
            # Train both models
            loss1 = self._train_epoch_warmup(self.model1, self.optimizer1, dataloader)
            loss2 = self._train_epoch_warmup(self.model2, self.optimizer2, dataloader)
            
            avg_loss = (loss1 + loss2) / 2
            self.train_history['loss'].append(avg_loss)
            
            # Validation
            if val_dataloader is not None:
                self.validate_models(val_dataloader, epoch)
            
            if epoch % 5 == 0:
                print(f"Warmup epoch {epoch}/{epochs} - Train Loss: {avg_loss:.4f}")
    
    def _train_epoch_warmup(self, model, optimizer, dataloader):
        """Single warmup epoch with symmetric cross-entropy"""
        model.train()
        total_loss = 0
        
        for batch in dataloader:
            batch = batch.to(self.device)
            
            optimizer.zero_grad()
            logits = model(batch)
            
            # Symmetric Cross-Entropy Loss
            ce_loss = F.cross_entropy(logits, batch.y, reduction='none')
            
            # Reverse cross-entropy
            pred_softmax = F.softmax(logits, dim=1)
            pred_softmax = torch.clamp(pred_softmax, min=1e-7, max=1.0)
            label_one_hot = F.one_hot(batch.y, logits.shape[1]).float()
            rce_loss = -torch.sum(pred_softmax * torch.log(label_one_hot + 1e-7), dim=1)
            
            loss = (0.1 * ce_loss + 1.0 * rce_loss).mean()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        return total_loss / len(dataloader)
    
    def divide_samples(self, model, dataloader, clean_ratio=0.5):
        """Divide samples into clean and noisy based on small loss"""
        model.eval()
        all_losses = []
        all_indices = []
        
        with torch.no_grad():
            global_idx = 0
            for batch in dataloader:
                batch = batch.to(self.device)
                logits = model(batch)
                losses = F.cross_entropy(logits, batch.y, reduction='none')
                
                for i, loss in enumerate(losses):
                    all_losses.append(loss.item())
                    all_indices.append(global_idx + i)
                
                global_idx += len(batch.y)
        
        # Sort by loss and take clean_ratio as clean samples
        sorted_indices = sorted(zip(all_indices, all_losses), key=lambda x: x[1])
        num_clean = int(len(sorted_indices) * clean_ratio)
        
        clean_indices = [idx for idx, _ in sorted_indices[:num_clean]]
        noisy_indices = [idx for idx, _ in sorted_indices[num_clean:]]
        
        return clean_indices, noisy_indices
    
    def mixup_data(self, batch1, batch2, alpha=0.75):
        """
        Alternative MixUp for graph data - uses label mixing only
        This avoids the complexity of mixing variable-sized graph structures
        """
        if alpha > 0:
            lam = np.random.beta(alpha, alpha)
        else:
            lam = 1
        
        # Determine batch size for mixing
        batch_size = min(len(batch1.y), len(batch2.y))
        
        # Sample indices from both batches
        indices1 = torch.randperm(len(batch1.y))[:batch_size]
        indices2 = torch.randperm(len(batch2.y))[:batch_size]
        
        # Create mixed labels using selected samples
        y1_selected = batch1.y[indices1]
        y2_selected = batch2.y[indices2]
        
        y1_one_hot = F.one_hot(y1_selected, num_classes=self.model1.num_class).float()
        y2_one_hot = F.one_hot(y2_selected, num_classes=self.model1.num_class).float()
        mixed_targets = lam * y1_one_hot + (1 - lam) * y2_one_hot
        
        # Return the first batch for forward pass, but with mixed supervision
        return batch1, mixed_targets, lam, indices1
    
    def graph_feature_mixup(self, batch1, batch2, alpha=0.75):
        """
        Advanced MixUp that handles graph structure mixing
        Only use this if your graphs have similar structures
        """
        if alpha > 0:
            lam = np.random.beta(alpha, alpha)
        else:
            lam = 1
        
        # Check if we can mix at node level (similar graph sizes)
        if abs(batch1.x.size(0) - batch2.x.size(0)) / max(batch1.x.size(0), batch2.x.size(0)) < 0.1:
            # Graphs are similar size, can mix node features
            min_nodes = min(batch1.x.size(0), batch2.x.size(0))
            
            # Truncate to same size
            mixed_x = lam * batch1.x[:min_nodes] + (1 - lam) * batch2.x[:min_nodes]
            
            # Create new batch
            mixed_batch = batch1.clone()
            mixed_batch.x = mixed_x[:min_nodes]
            
            # Adjust other batch attributes if needed
            if hasattr(mixed_batch, 'batch'):
                # Truncate batch indices
                valid_batch_mask = mixed_batch.batch < min_nodes
                mixed_batch.batch = mixed_batch.batch[valid_batch_mask]
            
            # Mix labels
            min_graphs = min(len(batch1.y), len(batch2.y))
            y1_one_hot = F.one_hot(batch1.y[:min_graphs], num_classes=self.model1.num_class).float()
            y2_one_hot = F.one_hot(batch2.y[:min_graphs], num_classes=self.model1.num_class).float()
            mixed_targets = lam * y1_one_hot + (1 - lam) * y2_one_hot
            
            return mixed_batch, mixed_targets, lam
        else:
            # Fall back to label-only mixing
            return self.mixup_data(batch1, batch2, alpha)
    
    def train_dividemix(self, dataloader, val_dataloader=None, epochs=100, clean_ratio_schedule=None):
        """Main DivideMix training loop with validation and BatchNorm safety"""
        if clean_ratio_schedule is None:
            # More conservative schedule for high noise (70%) and BatchNorm safety
            clean_ratio_schedule = [0.25 + 0.25 * epoch / epochs for epoch in range(epochs)]
        
        # Warmup training
        self.warmup_training(dataloader, val_dataloader, self.warmup_epochs)
        
        print("Starting DivideMix training...")
        for epoch in range(epochs):
            current_clean_ratio = clean_ratio_schedule[epoch]
            
            # Divide samples using both models
            clean_indices1, noisy_indices1 = self.divide_samples(
                self.model2, dataloader, current_clean_ratio)
            clean_indices2, noisy_indices2 = self.divide_samples(
                self.model1, dataloader, current_clean_ratio)
            
            # Check if we have sufficient samples for BatchNorm
            min_samples_needed = 8  # Minimum for stable BatchNorm
            
            if len(clean_indices1) < min_samples_needed:
                print(f"Warning: Too few clean samples for model1 ({len(clean_indices1)}), increasing clean ratio")
                current_clean_ratio = min(0.8, current_clean_ratio + 0.1)
                clean_indices1, noisy_indices1 = self.divide_samples(
                    self.model2, dataloader, current_clean_ratio)
                    
            if len(clean_indices2) < min_samples_needed:
                print(f"Warning: Too few clean samples for model2 ({len(clean_indices2)}), increasing clean ratio")
                current_clean_ratio = min(0.8, current_clean_ratio + 0.1)
                clean_indices2, noisy_indices2 = self.divide_samples(
                    self.model1, dataloader, current_clean_ratio)
            
            # Create dataloaders for clean and noisy samples with BatchNorm safety
            clean_loader1 = self._create_subset_loader(clean_indices1, shuffle=True, min_batch_size=2)
            noisy_loader1 = self._create_subset_loader(noisy_indices1, shuffle=True, min_batch_size=2)
            clean_loader2 = self._create_subset_loader(clean_indices2, shuffle=True, min_batch_size=2)
            noisy_loader2 = self._create_subset_loader(noisy_indices2, shuffle=True, min_batch_size=2)
            
            # Train model1
            loss1 = self._train_epoch_dividemix(
                self.model1, self.optimizer1, clean_loader1, noisy_loader1, clean_loader2)
            
            # Train model2
            loss2 = self._train_epoch_dividemix(
                self.model2, self.optimizer2, clean_loader2, noisy_loader2, clean_loader1)
            
            # Track training loss
            avg_loss = (loss1 + loss2) / 2
            self.train_history['loss'].append(avg_loss)
            
            # Validation
            if val_dataloader is not None:
                val_acc = self.validate_models(val_dataloader, self.warmup_epochs + epoch)
                
                # Early stopping check
                if self.should_early_stop():
                    print(f"Early stopping at epoch {epoch}")
                    break
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Clean ratio = {current_clean_ratio:.3f}, "
                      f"Clean1: {len(clean_indices1)}, Clean2: {len(clean_indices2)}, "
                      f"Train Loss: {avg_loss:.4f}")
        
        # Load best models at the end
        if val_dataloader is not None:
            self.load_best_models()
            print(f"Loaded best models with validation accuracy: {self.best_val_acc:.4f}")
    
    def _create_subset_loader(self, indices, batch_size=32, shuffle=False, min_batch_size=2):
        """Create dataloader for subset of indices with minimum batch size for BatchNorm"""
        if len(indices) < min_batch_size:
            return []  # Return empty if not enough samples for BatchNorm
        
        subset = Subset(self.dataset, indices)
        
        # Adjust batch size if subset is too small
        effective_batch_size = min(batch_size, len(indices))
        if effective_batch_size < min_batch_size:
            effective_batch_size = min_batch_size
        
        return DataLoader(subset, batch_size=effective_batch_size, shuffle=shuffle, drop_last=True, num_workers=8, pin_memory=True)
    
    def _train_epoch_dividemix(self, model, optimizer, clean_loader, noisy_loader, other_clean_loader):
        """Single epoch of DivideMix training with BatchNorm safety checks"""
        model.train()
        total_loss = 0
        batch_count = 0
        
        # Check if we have valid loaders (not empty and sufficient samples)
        has_clean = clean_loader and len(clean_loader) > 0
        has_noisy = noisy_loader and len(noisy_loader) > 0
        has_other_clean = other_clean_loader and len(other_clean_loader) > 0
        
        if not has_clean and not has_noisy:
            print("Warning: No valid batches for training (insufficient samples for BatchNorm)")
            return 0.0
        
        # Convert to iterators for parallel iteration
        clean_iter = iter(clean_loader) if has_clean else None
        noisy_iter = iter(noisy_loader) if has_noisy else None
        other_clean_iter = iter(other_clean_loader) if has_other_clean else None
        
        max_batches = max(len(clean_loader) if has_clean else 0,
                         len(noisy_loader) if has_noisy else 0)
        
        if max_batches == 0:
            return 0.0
        
        for batch_idx in range(max_batches):
            optimizer.zero_grad()
            batch_loss = 0
            
            # Supervised loss on clean samples
            if has_clean:
                try:
                    clean_batch = next(clean_iter)
                    clean_batch = clean_batch.to(self.device)
                    
                    # Safety check for batch size
                    if len(clean_batch.y) >= 2:
                        clean_logits = model(clean_batch)
                        clean_loss = F.cross_entropy(clean_logits, clean_batch.y)
                        batch_loss += clean_loss
                        
                except StopIteration:
                    clean_iter = iter(clean_loader)
                    try:
                        clean_batch = next(clean_iter)
                        clean_batch = clean_batch.to(self.device)
                        if len(clean_batch.y) >= 2:
                            clean_logits = model(clean_batch)
                            clean_loss = F.cross_entropy(clean_logits, clean_batch.y)
                            batch_loss += clean_loss
                    except StopIteration:
                        pass
            
            # Semi-supervised loss on noisy samples with MixUp
            if has_noisy and has_other_clean:
                try:
                    noisy_batch = next(noisy_iter)
                    other_clean_batch = next(other_clean_iter)
                    
                    noisy_batch = noisy_batch.to(self.device)
                    other_clean_batch = other_clean_batch.to(self.device)
                    
                    # Safety check for batch sizes
                    if len(noisy_batch.y) >= 1 and len(other_clean_batch.y) >= 1:
                        # Apply MixUp between noisy and clean samples
                        mixed_batch, mixed_targets, lam, selected_indices = self.mixup_data(noisy_batch, other_clean_batch)
                        mixed_batch = mixed_batch.to(self.device)
                        
                        # Additional safety check for the mixed batch
                        if len(selected_indices) >= 2:
                            mixed_logits = model(mixed_batch)
                            # Only use the selected subset for loss calculation
                            mixed_logits_subset = mixed_logits[selected_indices]
                            mixed_loss = -torch.sum(F.log_softmax(mixed_logits_subset, dim=1) * mixed_targets, dim=1).mean()
                            
                            batch_loss += 0.5 * mixed_loss  # Weight the semi-supervised loss
                        
                except StopIteration:
                    # Reset iterators
                    if has_noisy:
                        noisy_iter = iter(noisy_loader)
                    if has_other_clean:
                        other_clean_iter = iter(other_clean_loader)
            
            if batch_loss > 0:
                batch_loss.backward()
                optimizer.step()
                total_loss += batch_loss.item()
                batch_count += 1
        
        return total_loss / max(batch_count, 1)
    
    def get_final_clean_samples(self, dataloader, final_clean_ratio=0.5):
        """Get final set of clean samples using both models"""
        # Adjusted for high noise rate
        clean1, _ = self.divide_samples(self.model1, dataloader, final_clean_ratio)
        clean2, _ = self.divide_samples(self.model2, dataloader, final_clean_ratio)
        
        # Take intersection for highest confidence
        clean_intersection = list(set(clean1) & set(clean2))
        
        # If intersection is too small, take union (adjusted threshold for high noise)
        if len(clean_intersection) < len(self.dataset) * 0.15:  # Lower threshold for 70% noise
            clean_final = list(set(clean1) | set(clean2))
        else:
            clean_final = clean_intersection
        
        return clean_final
    
    def plot_training_history(self):
        """Plot training and validation curves"""
        import matplotlib.pyplot as plt
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        # Loss plot
        axes[0].plot(self.train_history['loss'], label='Train Loss')
        if self.val_history['loss']:
            axes[0].plot(self.val_history['loss'], label='Val Loss')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].set_title('Training and Validation Loss')
        axes[0].legend()
        
        # Accuracy plot
        if self.val_history['accuracy']:
            axes[1].plot(self.val_history['accuracy'], label='Val Accuracy')
            axes[1].plot(self.val_history['f1'], label='Val F1')
            axes[1].set_xlabel('Epoch')
            axes[1].set_ylabel('Score')
            axes[1].set_title('Validation Metrics')
            axes[1].legend()
        
        plt.tight_layout()
        plt.show()

In [97]:
# Create two identical models
model1 = GNNContrastive(
    gnn_type='gin',
    num_class=6,
    num_layer=args.num_layer,
    emb_dim=args.emb_dim,
    drop_ratio=args.drop_ratio,
    virtual_node=True,
    residual=True,
).to(device)
model2 = GNNContrastive(
    gnn_type='gin',
    num_class=6,
    num_layer=args.num_layer,
    emb_dim=args.emb_dim,
    drop_ratio=args.drop_ratio,
    virtual_node=True,
    residual=True,
).to(device)

# Initialize trainer with validation
trainer = DivideMixGraphTrainer(model1, model2, train_dataset, val_dataset)

# Train with DivideMix and validation
trainer.train_dividemix(train_loader, val_loader, epochs=100)

# Get final clean samples (conservative for 70% noise)
clean_indices = trainer.get_final_clean_samples(train_loader, final_clean_ratio=0.4)

print(f"Final clean dataset size: {len(clean_indices)} / {len(dataset)}")

# Final evaluation on validation set
if val_loader:
    val_loss1, val_acc1, val_f1_1 = trainer.evaluate_model(trainer.model1, val_loader)
    val_loss2, val_acc2, val_f1_2 = trainer.evaluate_model(trainer.model2, val_loader)
    
    print(f"Final Model 1 - Val Acc: {val_acc1:.4f}, Val F1: {val_f1_1:.4f}")
    print(f"Final Model 2 - Val Acc: {val_acc2:.4f}, Val F1: {val_f1_2:.4f}")
    print(f"Ensemble Average - Val Acc: {(val_acc1 + val_acc2)/2:.4f}")

# Create clean dataloader for further training if needed
clean_loader__ = DataLoader(Subset(train_dataset, clean_indices), batch_size=32, shuffle=True)

# Plot training history
trainer.plot_training_history()

Warmup training phase...
Epoch 0 - Val Loss: 7.5826, Val Acc: 0.3147, Val F1: 0.2236
Warmup epoch 0/1 - Train Loss: 12.4585
Starting DivideMix training...
Epoch 1 - Val Loss: 1.8007, Val Acc: 0.3464, Val F1: 0.2882
Epoch 0: Clean ratio = 0.250, Clean1: 1120, Clean2: 1120, Train Loss: 3.7356
Epoch 2 - Val Loss: 1.7723, Val Acc: 0.3379, Val F1: 0.2922
Epoch 3 - Val Loss: 1.7219, Val Acc: 0.3598, Val F1: 0.3038
Epoch 4 - Val Loss: 1.7031, Val Acc: 0.3866, Val F1: 0.3225
Epoch 5 - Val Loss: 1.6563, Val Acc: 0.3790, Val F1: 0.3118
Epoch 6 - Val Loss: 1.6741, Val Acc: 0.3763, Val F1: 0.3198
Epoch 7 - Val Loss: 1.6367, Val Acc: 0.3915, Val F1: 0.3462
Epoch 8 - Val Loss: 1.7155, Val Acc: 0.3991, Val F1: 0.3388
Epoch 9 - Val Loss: 1.5964, Val Acc: 0.4165, Val F1: 0.3701
Epoch 10 - Val Loss: 1.6412, Val Acc: 0.4062, Val F1: 0.3516
Epoch 11 - Val Loss: 1.6551, Val Acc: 0.3969, Val F1: 0.3524
Epoch 10: Clean ratio = 0.275, Clean1: 1232, Clean2: 1232, Train Loss: 2.4914
Epoch 12 - Val Loss: 1.6275,

NameError: name 'dataset' is not defined

In [98]:
print(f"Final clean dataset size: {len(clean_indices)} / {len(train_dataset)}")

# Final evaluation on validation set
if val_loader:
    val_loss1, val_acc1, val_f1_1 = trainer.evaluate_model(trainer.model1, val_loader)
    val_loss2, val_acc2, val_f1_2 = trainer.evaluate_model(trainer.model2, val_loader)
    
    print(f"Final Model 1 - Val Acc: {val_acc1:.4f}, Val F1: {val_f1_1:.4f}")
    print(f"Final Model 2 - Val Acc: {val_acc2:.4f}, Val F1: {val_f1_2:.4f}")
    print(f"Ensemble Average - Val Acc: {(val_acc1 + val_acc2)/2:.4f}")

# Create clean dataloader for further training if needed
# clean_loader__ = DataLoader(Subset(train_dataset, clean_indices), batch_size=32, shuffle=True)

# Plot training history


Final clean dataset size: 700 / 4480
Final Model 1 - Val Acc: 0.4750, Val F1: 0.4542
Final Model 2 - Val Acc: 0.4500, Val F1: 0.4459
Ensemble Average - Val Acc: 0.4625


In [99]:
val_loss, val_acc = evaluate(val_loader, trainer.model1, device, calculate_accuracy=True)
print(f"Val loss: {val_loss:.4f}, Val Acc {val_acc:.4f}")

Iterating eval graphs: 100%|██████████| 35/35 [00:03<00:00,  9.41batch/s]

Val loss: 0.0000, Val Acc 0.4750


In [100]:
clean_val_loss, clean_val_acc = evaluate(clean_val, trainer.model1, device, calculate_accuracy=True)
print(f"Val loss: {clean_val_loss:.4f}, Val Acc {clean_val_acc:.4f}")

Iterating eval graphs: 100%|██████████| 11/11 [00:03<00:00,  3.53batch/s]

Val loss: 0.0000, Val Acc 0.9048


In [57]:
checkpoint_file = f"{checkpoint_path}_pretrained.pth"
torch.save(model.state_dict(), checkpoint_file)
print(f"Checkpoint saved at {checkpoint_file}")

Checkpoint saved at /workspace/hackaton/checkpoints/model_B_best.pth_pretrained.pth


In [59]:
trainer.train_with_noise_robustness(clean_train, epochs=30)

Train Epoch 0, Loss: 10.4193
Train Epoch 10, Loss: 0.6879
Train Epoch 20, Loss: 0.6180


In [51]:
test_dataset = GraphDatasetWithFeatures(args.test_path)

In [52]:
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=8)

In [91]:
checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")

In [173]:
model.load_state_dict(torch.load(checkpoint_path))

<All keys matched successfully>

In [101]:
predictions = evaluate(test_loader, trainer.model1, device, calculate_accuracy=False)

Iterating eval graphs: 100%|██████████| 49/49 [00:04<00:00, 10.72batch/s]


In [102]:
save_predictions(predictions, args.test_path)

Predictions saved to /workspace/hackaton/submission/testset_B.csv


In [116]:
# best Epoch 47/50, Loss: 0.4592, Val loss: 0.5975, Train Acc: 0.7461, Val Acc 0.6941
# needs retrain

# num_layer=5,
# emb_dim=300,
# noize 0.3
# without training on the full dataset

# optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-4)
# scheduler = StepLR(optimizer, step_size=10, gamma=0.8)

In [98]:
!pwd

/workspace/hackaton


In [103]:
import tarfile
import os

In [104]:
def gzip_folder(folder_path, output_file):
    """
    Compresses an entire folder into a single .tar.gz file.
    
    Args:
        folder_path (str): Path to the folder to compress.
        output_file (str): Path to the output .tar.gz file.
    """
    with tarfile.open(output_file, "w:gz") as tar:
        tar.add(folder_path, arcname=os.path.basename(folder_path))
    print(f"Folder '{folder_path}' has been compressed into '{output_file}'")
    

# Example usage
folder_path = "./submission"            # Path to the folder you want to compress
output_file = "./submission.gz"         # Output .gz file name
gzip_folder(folder_path, output_file)

Folder './submission' has been compressed into './submission.gz'
